In [21]:
import numpy as np
import pandas as pd
import yfinance as yf

from scipy.optimize import minimize

In [78]:
pd.set_option('display.max_rows', 20)

In [22]:
import yfinance as yf
import pandas as pd
import openpyxl
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from scipy import optimize

import os
cwd = os.getcwd()
print(cwd)
os.chdir(r'C:\Users\mitch\Documents\QCF\Finance and Investments\StockTrack-Trading-Simulation')

C:\Users\mitch\Documents\QCF\Finance and Investments\StockTrack-Trading-Simulation


# Reusable functions

In [23]:
def calculate_return(df):
    if df.empty:
        return None
    # Get the closing price at the start of the year
    start_price = df['Close'].iloc[0]
    
    # Get the closing price at the end of the year
    end_price = df['Close'].iloc[-1]
    
    # Calculate the one-year return
    return (end_price - start_price) / start_price

def MaximizeSharpeRatioOptmzn(MeanReturns, CovarReturns, RiskFreeRate, PortfolioSize):
     # define maximization of Sharpe Ratio using principle of duality
    def  f(x, MeanReturns, CovarReturns, RiskFreeRate, PortfolioSize):
        funcDenomr = np.sqrt(np.matmul(np.matmul(x, CovarReturns), x.T) )
        funcNumer = np.matmul(np.array(MeanReturns),x.T)-RiskFreeRate
        func = -(funcNumer / funcDenomr)
        return func

    #define equality constraint representing fully invested portfolio
    def constraintEq(x):
        A=np.ones(x.shape)
        b=1
        constraintVal = np.matmul(A,x.T)-b 
        return constraintVal
    
    #define bounds and other parameters
    xinit=np.repeat(0.33, PortfolioSize)
    cons = ({'type': 'eq', 'fun':constraintEq})
    lb = 0
    ub = 1
    bnds = tuple([(lb,ub) for x in xinit])
    
    #invoke minimize solver
    opt = optimize.minimize (f, x0 = xinit, args = (MeanReturns, CovarReturns,\
                             RiskFreeRate, PortfolioSize), method = 'SLSQP',  \
                             bounds = bnds, constraints = cons, tol = 10**-3)
    
    return opt
    

In [24]:
largest_market_cap = pd.read_excel('./Top Companies by Market Cap.xlsx', sheet_name='Sheet1')
largest_market_cap

,Symbol,Company Name,Industry,Market Cap
0,AAPL,Apple Inc.,Consumer Electronics,"2,811.03B"
1,MSFT,Microsoft Corporation,Software - Infrastructure,"2,469.80B"
2,GOOGL,Alphabet Inc.,Internet Content & Information,"1,779.01B"
3,GOOG,Alphabet Inc.,Internet Content & Information,"1,778.79B"
4,AMZN,"Amazon.com, Inc.",Internet Retail,"1,360.20B"
...,...,...,...,...
495,RBLX,Roblox Corporation,Electronic Gaming & Multimedia,19.59B
496,WBA,"Walgreens Boots Alliance, Inc.",Pharmaceutical Retailers,19.51B
497,TW,Tradeweb Markets Inc.,Capital Markets,19.51B
498,ZM,"Zoom Video Communications, Inc.",Software - Application,19.51B


In [25]:
largest_market_cap = largest_market_cap[~largest_market_cap['Symbol'].str.contains('\.')].reset_index(drop=True)
top_300 = largest_market_cap.head(100)
top_300_symbols = top_300['Symbol']

In [26]:
# ticker = yf.Ticker('AAPL')
# stock_info = ticker.info
# print(stock_info.get('beta'))

In [27]:
one_year_data = {}
three_year_data = {}
five_year_data = {} 
for symbol in top_300_symbols:
    ticker = yf.Ticker(symbol)
    one_year_data[symbol] = ticker.history(period="1y")
    three_year_data[symbol] = ticker.history(period="3y")
    five_year_data[symbol] = ticker.history(period="5y")

In [28]:
one_year_return_data = {}
three_year_return_data = {}
five_year_return_data = {}
for key, df in one_year_data.items():
    one_year_return_data[key] = calculate_return(one_year_data[key])
    three_year_return_data[key] = calculate_return(three_year_data[key])
    five_year_return_data[key] = calculate_return(five_year_data[key])
    
one_year_return_data

{'AAPL': 0.2532569689154102,
 'MSFT': 0.53269479182366,
 'GOOGL': 0.38025078944704027,
 'GOOG': 0.3916484545291894,
 'AMZN': 0.4477611882489815,
 'NVDA': 1.985562512022651,
 'META': 1.8820696806448503,
 'TSLA': 0.17156329033356696,
 'LLY': 0.7345739174097993,
 'V': 0.19217314239362615,
 'UNH': 0.07164587129098937,
 'TSM': 0.35147720281661615,
 'NVO': 0.8071784164159003,
 'WMT': 0.2301974728119171,
 'JPM': 0.12109221626815235,
 'XOM': -0.052332967298438345,
 'JNJ': -0.11654101868415831,
 'MA': 0.16909596882739106,
 'AVGO': 0.8941050721895167,
 'PG': 0.10888360395586448,
 'CVX': -0.19994079151670618,
 'ORCL': 0.4756670123596331,
 'HD': -0.03593613990157222,
 'ABBV': -0.04942156536880995,
 'MRK': 0.04905661084746388,
 'COST': 0.14308722185820147,
 'ADBE': 0.7344067788707931,
 'TM': 0.3209964251572046,
 'ASML': 0.14417260642599244,
 'KO': -0.03383187323422096,
 'PEP': -0.024791635770954744,
 'BABA': 0.15323145975887562,
 'SHEL': 0.2442411052018249,
 'CSCO': 0.20319611329451912,
 'BAC': -0.

In [29]:
def get_deciles(data):
    # Remove None values and sort the data
    filtered_data = {k: v for k, v in data.items() if v is not None}
    sorted_data = sorted(filtered_data.items(), key=lambda x: x[1])

    # Calculate the index for decile
    n = len(sorted_data)
    decile_index = n // 10

    # Get the top and bottom decile
    bottom_decile = dict(sorted_data[:decile_index])
    top_decile = dict(sorted_data[-decile_index:])

    return top_decile, bottom_decile

# Apply the function to each dataset
top_decile_one_year, bottom_decile_one_year = get_deciles(one_year_return_data)
top_decile_three_year, bottom_decile_three_year = get_deciles(three_year_return_data)
top_decile_five_year, bottom_decile_five_year = get_deciles(five_year_return_data)

# Find overlapping keys in the top deciles
overlapping_top_keys = set(top_decile_one_year.keys()) & set(top_decile_three_year.keys()) & set(top_decile_five_year.keys())

# Find overlapping keys in the bottom deciles
overlapping_bottom_keys = set(bottom_decile_one_year.keys()) & set(bottom_decile_three_year.keys()) & set(bottom_decile_five_year.keys())

# Print results
print("Top Decile Overlaps:", overlapping_top_keys)
print("Bottom Decile Overlaps:", overlapping_bottom_keys)


Top Decile Overlaps: {'NVDA', 'LLY', 'NVO'}
Bottom Decile Overlaps: set()


# Choosing top-50 and bottom-50 companies

In [30]:
'''''

target_long = pd.DataFrame(['ASML', 'GOOG', 'ACN', 'AAPL', 'SONY',
                            'INTU', 'V', 'MA', 'BX', 'TTE'],
                           columns=['Ticker'])
# TODO
target_short = pd.DataFrame([],
                            columns=['Ticker']
                            )
                            
                            '''''

"''\n\ntarget_long = pd.DataFrame(['ASML', 'GOOG', 'ACN', 'AAPL', 'SONY',\n                            'INTU', 'V', 'MA', 'BX', 'TTE'],\n                           columns=['Ticker'])\n# TODO\ntarget_short = pd.DataFrame([],\n                            columns=['Ticker']\n                            )\n                            \n                            "

In [39]:
# Get all unique keys in the top deciles
all_top_keys = set(top_decile_one_year.keys()) | set(top_decile_three_year.keys()) | set(top_decile_five_year.keys())

# Get all unique keys in the bottom deciles
all_bottom_keys = set(bottom_decile_one_year.keys()) | set(bottom_decile_three_year.keys()) | set(bottom_decile_five_year.keys())


all_top_keys

{'AAPL',
 'ADBE',
 'AMAT',
 'AMD',
 'AMGN',
 'ASML',
 'AVGO',
 'BP',
 'BX',
 'COP',
 'FMX',
 'GE',
 'LLY',
 'META',
 'NOW',
 'NVDA',
 'NVO',
 'PDD',
 'TSLA',
 'WFC'}

In [40]:
target_long = pd.DataFrame(list(all_top_keys),
                           columns=['Ticker'])
# TODO
target_short = pd.DataFrame(list(all_bottom_keys),
                            columns=['Ticker'])


# Getting 2-month history

In [41]:
def get_history(target_df: pd.DataFrame, long=True) -> pd.DataFrame:
    history = pd.DataFrame(columns=list(target_long['Ticker']))
    for ticker in target_df['Ticker']:
        # TODO think about the appropriate price column to use
        history[ticker] = yf.Ticker(ticker).history(interval='5m')['High' if long else 'Low']
    return history

In [42]:
history_long = get_history(target_long)
history_long.info()
history_long.head()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1638 entries, 2023-10-16 09:30:00-04:00 to 2023-11-13 15:55:00-05:00
Data columns (total 20 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   NOW     1638 non-null   float64
 1   NVO     1638 non-null   float64
 2   BP      1638 non-null   float64
 3   COP     1638 non-null   float64
 4   FMX     1636 non-null   float64
 5   LLY     1638 non-null   float64
 6   AVGO    1638 non-null   float64
 7   GE      1638 non-null   float64
 8   AMAT    1638 non-null   float64
 9   ASML    1637 non-null   float64
 10  ADBE    1638 non-null   float64
 11  AMD     1638 non-null   float64
 12  META    1638 non-null   float64
 13  BX      1638 non-null   float64
 14  NVDA    1638 non-null   float64
 15  WFC     1638 non-null   float64
 16  AAPL    1638 non-null   float64
 17  TSLA    1638 non-null   float64
 18  PDD     1638 non-null   float64
 19  AMGN    1638 non-null   float64
dtypes: float64(20)
memory usag

,NOW,NVO,BP,COP,FMX,LLY,AVGO,GE,AMAT,ASML,ADBE,AMD,META,BX,NVDA,WFC,AAPL,TSLA,PDD,AMGN
Datetime,,,,,,,,,,,,,,,,,,,,
2023-10-16 09:30:00-04:00,550.299988,101.419998,40.185001,126.120003,106.169998,617.000000,906.929993,110.125000,142.199997,603.429993,554.369995,106.379303,318.859985,105.470001,454.829987,41.680000,178.179993,253.639999,105.910004,286.334991
2023-10-16 09:35:00-04:00,553.799988,101.320000,40.134998,125.809998,106.364998,615.414978,912.849976,110.400002,142.654999,606.049988,554.969971,107.014999,318.959991,105.044998,458.489990,41.910000,178.360001,254.169998,105.000000,286.480011
2023-10-16 09:40:00-04:00,552.630005,101.239998,40.090000,125.599998,106.190002,611.770020,915.784973,110.089996,142.429993,605.429993,555.539978,107.120003,318.500000,104.720001,458.600006,42.165001,178.322495,253.419998,104.930000,287.309998
2023-10-16 09:45:00-04:00,548.229980,101.150002,40.090000,126.010002,106.187500,612.388428,907.340515,109.830002,141.369995,601.510010,551.789978,106.459900,317.179901,104.269997,456.629791,42.130001,177.509995,252.899994,105.267700,287.309998
2023-10-16 09:50:00-04:00,549.210022,101.099998,40.115002,126.049400,106.654999,612.419983,903.875000,109.940002,140.709900,599.650024,549.309998,106.209999,317.500000,104.580002,453.750000,42.070000,177.429993,252.370605,106.000000,286.579987


In [92]:
history_short = get_history(target_short)
history_short.info()
history_short.head()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1638 entries, 2023-10-16 09:30:00-04:00 to 2023-11-13 15:55:00-05:00
Data columns (total 40 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   NOW     0 non-null      object 
 1   NVO     0 non-null      object 
 2   BP      0 non-null      object 
 3   COP     0 non-null      object 
 4   FMX     0 non-null      object 
 5   LLY     0 non-null      object 
 6   AVGO    0 non-null      object 
 7   GE      0 non-null      object 
 8   AMAT    0 non-null      object 
 9   ASML    0 non-null      object 
 10  ADBE    0 non-null      object 
 11  AMD     0 non-null      object 
 12  META    0 non-null      object 
 13  BX      0 non-null      object 
 14  NVDA    0 non-null      object 
 15  WFC     1638 non-null   float64
 16  AAPL    0 non-null      object 
 17  TSLA    0 non-null      object 
 18  PDD     1638 non-null   float64
 19  AMGN    0 non-null      object 
 20  CRM     1638 non-null   fl

,NOW,NVO,BP,COP,FMX,LLY,AVGO,GE,AMAT,ASML,...,BABA,DHR,UPS,TXN,UL,NEE,NKE,BA,BAC,INTC
Datetime,,,,,,,,,,,,,,,,,,,,,
2023-10-16 09:30:00-04:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,83.709999,211.029999,156.770004,154.100006,47.689999,54.799999,101.110001,186.229996,27.180000,36.259998
2023-10-16 09:35:00-04:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,83.019897,210.559998,156.929993,154.380005,47.660000,54.080002,101.190002,185.630997,27.070000,36.445000
2023-10-16 09:40:00-04:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,83.290001,209.625000,157.089996,154.360001,47.529999,54.174999,101.150002,184.770004,27.040001,36.580002
2023-10-16 09:45:00-04:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,83.730003,209.740005,157.083298,153.789993,47.570000,54.520000,101.510002,184.250000,27.014999,36.580002
2023-10-16 09:50:00-04:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,84.220001,210.119995,157.369995,153.810699,47.599998,54.560001,101.989998,183.809998,27.010000,36.634998


# Getting 2-month compound returns

In [53]:
def get_returns(history_df: pd.DataFrame) -> np.ndarray:
    n = len(history_df.columns)
    returns = np.zeros(n, dtype=float)
    for i in range(n):
        # TODO think about the appropriate return calculation
        helper = history_df.iloc[:, i] / history_df.iloc[:, i].shift(1)
        helper = helper.iloc[1:]
        returns[i] = np.prod(helper) - 1
    return returns

In [54]:
returns_long = get_returns(history_long)
print(f'Top companies\' returns: {returns_long}')

Top companies' returns: [ 0.16211162 -0.00507789 -0.11160882 -0.08499842  0.13437297 -0.00445705
  0.04431434  0.04953464  0.05362166  0.08578396  0.06517309  0.09875698
  0.03405889 -0.07205839  0.07133195 -0.02087329  0.03709732 -0.11677967
  0.03389667 -0.06763054]


In [55]:
returns_short = get_returns(history_short)
print(f'Bottom companies\' returns: {returns_short}')

Bottom companies' returns: [ 0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.         -0.02087329  0.          0.
  0.03389667  0.          0.04602163 -0.12906736  0.10820583  0.0770826
  0.05609302 -0.04053605  0.15959859 -0.13057798 -0.11923548 -0.08066081
 -0.01612709 -0.0712221  -0.11813487 -0.05243349  0.01300069 -0.00510947
  0.03214321  0.09939328  0.02023544  0.05529512]


# Getting weights

In [87]:
def sharpe_long(weights):
    return -np.dot(returns_long, weights) / np.sqrt(weights @ history_long.cov().to_numpy() @ weights)


def sharpe_short(weights):
    return np.dot(returns_short, weights) / np.sqrt(weights @ history_short.cov().to_numpy() @ weights)

In [88]:
'''''
def get_weights(history_df: pd.DataFrame, long=True) -> np.ndarray:
    n = len(history_df.columns)
    weights = np.ones(n)

    bounds = [(0.0, None)] * n if long else [(None, 0.0)] * n
    constraints = ({'type': 'eq', 'fun': lambda weights: weights.sum() - 1.0})
    optimal_weights = minimize(sharpe_long if long else sharpe_short,
                               weights, 
                               method='SLSQP',
                               constraints=constraints,
                               bounds=bounds
                               ).x

    return optimal_weights
    '''''

"''\ndef get_weights(history_df: pd.DataFrame, long=True) -> np.ndarray:\n    n = len(history_df.columns)\n    weights = np.ones(n)\n\n    bounds = [(0.0, None)] * n if long else [(None, 0.0)] * n\n    constraints = ({'type': 'eq', 'fun': lambda weights: weights.sum() - 1.0})\n    optimal_weights = minimize(sharpe_long if long else sharpe_short,\n                               weights, \n                               method='SLSQP',\n                               constraints=constraints,\n                               bounds=bounds\n                               ).x\n\n    return optimal_weights\n    "

In [89]:
def get_weights(history_df: pd.DataFrame, long=True) -> dict:
    n = len(history_df.columns)
    weights = np.ones(n)

    bounds = [(0.0, 0.10)] * n if long else [(None, 0.0)] * n
    constraints = ({'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1.0})
    optimal_weights = minimize(sharpe_long if long else sharpe_short,
                               weights, 
                               method='SLSQP',
                               constraints=constraints,
                               bounds=bounds
                               ).x

    # Create a dictionary with column names as keys and weights as values
    weights_dict = dict(zip(history_df.columns, optimal_weights))
    return weights_dict

In [90]:
weights_long = get_weights(history_long, True)
weights_long_vw = pd.DataFrame(list(weights_long.items()), columns=['Stock', 'Weight'])

weights_long_vw['Weight'] = weights_long_vw['Weight'].apply(lambda x: f"{x * 100:.2f}%")

#print(f'Optimal weights for top companies: {weights_long}')

#print(f'Optimal Sharpe ratio: {-sharpe_long(weights_long)}')

#history_long.columns

print(weights_long_vw)



   Stock  Weight
0    NOW   1.88%
1    NVO  10.00%
2     BP   0.00%
3    COP   2.89%
4    FMX  10.00%
5    LLY   0.00%
6   AVGO   0.00%
7     GE  10.00%
8   AMAT  10.00%
9   ASML   0.00%
10  ADBE   0.00%
11   AMD  10.00%
12  META  10.00%
13    BX   0.00%
14  NVDA   0.00%
15   WFC  10.00%
16  AAPL  10.00%
17  TSLA   0.00%
18   PDD  10.00%
19  AMGN   5.23%


In [91]:
weights_short = get_weights(history_short, False)
print(f'Optimal weights for bottom companies: {weights_short}')
print(f'Optimal Sharpe ratio: {-sharpe_short(np.array(list(weights_short.values())))}')

Optimal weights for bottom companies: {'NOW': 0.0, 'NVO': 0.0, 'BP': 0.0, 'COP': 0.0, 'FMX': 0.0, 'LLY': 0.0, 'AVGO': 0.0, 'GE': 0.0, 'AMAT': 0.0, 'ASML': 0.0, 'ADBE': 0.0, 'AMD': 0.0, 'META': 0.0, 'BX': 0.0, 'NVDA': 0.0, 'WFC': 0.0, 'AAPL': 0.0, 'TSLA': 0.0, 'PDD': 0.0, 'AMGN': 0.0, 'CRM': 0.0, 'CVX': 0.0, 'BUD': 0.0, 'T': 0.0, 'DIS': 0.0, 'HDB': 0.0, 'VZ': 0.0, 'PFE': 0.0, 'BMY': 0.0, 'TMO': 0.0, 'BABA': 0.0, 'DHR': 0.0, 'UPS': 0.0, 'TXN': 0.0, 'UL': 0.0, 'NEE': 0.0, 'NKE': 0.0, 'BA': 0.0, 'BAC': 0.0, 'INTC': 0.0}
Optimal Sharpe ratio: nan
